In [1]:
# NiN使用 1×1 卷积层来替代全连接层，从而使空间信息能够自然传递到后面的层中去

import d2lzh as d2l
from mxnet import gluon, init, nd
from mxnet.gluon import nn

In [2]:
# 定义NiN块
def nin_block(num_channels, kernel_size, strides, padding):
    blk = nn.Sequential()
    blk.add(nn.Conv2D(num_channels,kernel_size,
                      strides, padding, activation='relu'),
            nn.Conv2D(num_channels, kernel_size=1, activation='relu'),
            nn.Conv2D(num_channels, kernel_size=1, activation='relu'))
    return blk

In [3]:
# NiN还有一个设计与AlexNet显著不同：NiN去掉了AlexNet最后的3个全连接层，
# 取而代之地，NiN使用了输出通道数等于标签类别数的NiN块，然后使用全局平均池化层对每个通道中所有元素求平均并直接用于分类。
net = nn.Sequential()
net.add(nin_block(96, kernel_size=11, strides=4, padding=0),
        nn.MaxPool2D(pool_size=3, strides=2),
        nin_block(256, kernel_size=5, strides=1, padding=1),
        nn.MaxPool2D(pool_size=3, strides=2),
        nin_block(384, kernel_size=3, strides=1, padding=4),
        nn.MaxPool2D(pool_size=3, strides=2), nn.Dropout(0.5),
        # 标签类别数目为10
        nin_block(10, kernel_size=3, strides=1, padding=1),
        # 这里的全局平均池化层即窗口形状等于输入空间维形状的平均池化层，
        # 全局平均池化层将窗口形状自动设置成输入(即上一层输出)的高和宽
        nn.GlobalAvgPool2D(),
        # 将四维的输出转换为二维的输出，形状为(批量大小,10)
        nn.Flatten())

In [4]:
# 构建一个数据样本查看每一层的输出形状
X = nd.random.normal(shape=(1, 1, 224, 224))
net.initialize()
for layer in net:
    X = layer(X)
    print(layer.name, 'output shape:\t', X.shape)

sequential1 output shape:	 (1, 96, 54, 54)
pool0 output shape:	 (1, 96, 26, 26)
sequential2 output shape:	 (1, 256, 24, 24)
pool1 output shape:	 (1, 256, 11, 11)
sequential3 output shape:	 (1, 384, 17, 17)
pool2 output shape:	 (1, 384, 8, 8)
dropout0 output shape:	 (1, 384, 8, 8)
sequential4 output shape:	 (1, 10, 8, 8)
pool3 output shape:	 (1, 10, 1, 1)
flatten0 output shape:	 (1, 10)


In [ ]:
# 获取数据和训练模型
lr, num_epochs, batch_size, ctx = 0.1, 5, 128, d2l.try_gpu()
net.initialize(force_reinit=True, ctx=ctx, init=init.Xavier())
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224)
d2l.train_ch5(net, train_iter, test_iter, batch_size, trainer, ctx,
              num_epochs)